# Player Tracking Evluation

In [ ]:
import json
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from tqdm import tqdm


def bbox_area(bbox):
    x1, y1, x2, y2 = bbox
    return max(0, x2 - x1) * max(0, y2 - y1)


def bbox_center(bbox):
    x1, y1, x2, y2 = bbox
    return np.array([(x1 + x2) / 2, (y1 + y2) / 2])


def bbox_shape_ratio(bbox):
    x1, y1, x2, y2 = bbox
    w, h = max(1, x2 - x1), max(1, y2 - y1)
    return w / h


def angle_between(v1, v2):
    v1_u = v1 / np.linalg.norm(v1) if np.linalg.norm(v1) > 0 else np.zeros_like(v1)
    v2_u = v2 / np.linalg.norm(v2) if np.linalg.norm(v2) > 0 else np.zeros_like(v2)
    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    return np.degrees(angle)


def get_patch(frame, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    h, w = frame.shape[:2]
    x1 = max(0, min(w - 1, x1))
    x2 = max(0, min(w, x2))
    y1 = max(0, min(h - 1, y1))
    y2 = max(0, min(h, y2))
    return frame[y1:y2, x1:x2]


def histogram_similarity(patch1, patch2):
    h1 = cv2.calcHist([patch1], [0], None, [32], [0, 256])
    h2 = cv2.calcHist([patch2], [0], None, [32], [0, 256])
    h1, h2 = cv2.normalize(h1, h1).flatten(), cv2.normalize(h2, h2).flatten()
    return np.sum(np.minimum(h1, h2)) / np.sum(np.maximum(h1, h2))


def compute_fragmentation_metrics(track):
    frame_indices = sorted(t["frame_idx"] for t in track)
    gaps = np.diff(frame_indices)
    fragmentations = np.sum(gaps > 1)
    relative_fragmentation = fragmentations / len(frame_indices) if len(frame_indices) > 0 else 0
    return fragmentations, relative_fragmentation


def batchwise_frame_loader(cap, start_idx, end_idx):
    frames = {}
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_idx)
    for i in range(start_idx, end_idx):
        ret, frame = cap.read()
        if not ret:
            break
        frames[i] = frame
    return frames


def extract_player_tracks(tracking_data):
    player_tracks = defaultdict(list)
    for frame_idx, frame_data in tracking_data.items():
        for player in frame_data.get("players", []):
            pid = str(player["id"])
            player_tracks[pid].append({
                "frame_idx": int(frame_idx),
                "bbox": player["bbox"]
            })
    return player_tracks

def extract_ball_track(tracking_data):
    ball_track = []
    for frame_idx_str, frame_data in tracking_data.items():
        frame_idx = int(frame_idx_str)
        ball_entries = frame_data.get("ball", [])
        if ball_entries:
            bbox = ball_entries[0]["bbox"]
            ball_track.append({
                "frame_idx": frame_idx,
                "bbox": bbox
            })
    return ball_track


def evaluate_advanced_metrics_batchwise(tracking_log_path, video_path, output_dir="batch_eval", batch_size=200):
    os.makedirs(output_dir, exist_ok=True)
    output_excel_path = os.path.join(output_dir, "advanced_tracking_metrics_enhanced.xlsx")

    with open(tracking_log_path, "r") as f:
        tracking_data = json.load(f)

    total_frames = max(map(int, tracking_data.keys())) + 1
    player_tracks = extract_player_tracks(tracking_data)
    cap = cv2.VideoCapture(video_path)

    metrics = {
        "player_id": [], "histogram_similarity": [], "shape_ratio_consistency": [],
        "area_ratio_consistency": [], "relative_id_lifetime": [], "absolute_id_lifetime": [],
        "id_fragmentations": [], "relative_continuity": [], "overall_score": []
    }

    frame_progress = tqdm(total=total_frames, desc="Frames processed", unit="frame")

    for batch_start in range(0, total_frames, batch_size):
        batch_end = min(batch_start + batch_size, total_frames)
        frames = batchwise_frame_loader(cap, batch_start, batch_end)
        frame_progress.update(len(frames))

        for pid, track in player_tracks.items():
            if len(track) < 3:
                continue

            batch_track = [t for t in track if batch_start <= t["frame_idx"] < batch_end]
            if not batch_track:
                continue

            hist_sims, shape_ratios, area_ratios, centers = [], [], [], []

            for i in range(1, len(batch_track)):
                f1, f2 = batch_track[i - 1]["frame_idx"], batch_track[i]["frame_idx"]
                b1, b2 = batch_track[i - 1]["bbox"], batch_track[i]["bbox"]
                frame1, frame2 = frames.get(f1), frames.get(f2)
                if frame1 is None or frame2 is None:
                    continue

                patch1, patch2 = get_patch(frame1, b1), get_patch(frame2, b2)
                if patch1.size == 0 or patch2.size == 0:
                    continue

                try:
                    hist_sims.append(histogram_similarity(patch1, patch2))
                except:
                    continue

                center1 = bbox_center(b1)
                center2 = bbox_center(b2)
                centers.append(center2)

                shape_ratios.append(min(bbox_shape_ratio(b1), bbox_shape_ratio(b2)) /
                                    max(bbox_shape_ratio(b1), bbox_shape_ratio(b2)))
                area_ratios.append(min(bbox_area(b1), bbox_area(b2)) /
                                   max(bbox_area(b1), bbox_area(b2)))

            if not hist_sims:
                continue

            fragmentations, rel_fragmentation = compute_fragmentation_metrics(track)
            rel_continuity = 1 - rel_fragmentation
            rel_lifetime = len(track) / total_frames if total_frames > 0 else 0

            hist_sim = np.mean(hist_sims)
            shape_cons = np.mean(shape_ratios)
            area_cons = np.mean(area_ratios)
            overall_score = np.mean([hist_sim, shape_cons, area_cons, rel_lifetime, rel_continuity])

            metrics["player_id"].append(pid)
            metrics["histogram_similarity"].append(hist_sim)
            metrics["shape_ratio_consistency"].append(shape_cons)
            metrics["area_ratio_consistency"].append(area_cons)
            metrics["absolute_id_lifetime"].append(len(track))
            metrics["relative_id_lifetime"].append(rel_lifetime)
            metrics["id_fragmentations"].append(fragmentations)
            metrics["relative_continuity"].append(rel_continuity)
            metrics["overall_score"].append(overall_score)

    cap.release()
    frame_progress.close()

    df = pd.DataFrame(metrics)
    original_df = df.copy()
    original_avg_row = original_df.drop(columns=["player_id"]).mean().to_dict()
    original_avg_row["player_id"] = "AVERAGE"
    original_df = pd.concat([original_df, pd.DataFrame([original_avg_row])], ignore_index=True)

    with pd.ExcelWriter(output_excel_path, engine="openpyxl") as writer:
        original_df.to_excel(writer, sheet_name="Original", index=False)

    return original_df


In [ ]:
output_dir = "player_evaluation"
original_df = evaluate_advanced_metrics_batchwise("tracking_log.json", "input.mp4", output_dir)

for name, plot_df, color, title, filename in [
    ("Original", original_df, "lightblue", "Distribution of metrics for tracking and re-identification quality", "boxplot_all_metrics.png"),
]:
    plot_df = plot_df.drop(columns=["absolute_id_lifetime", "id_fragmentations", "player_id"])
    plot_df = plot_df.rename(columns={
        "histogram_similarity": "Histogram Similarity",
        "shape_ratio_consistency": "Shape Consistency",
        "area_ratio_consistency": "Area Consistency",
        "relative_id_lifetime": "Relative ID Lifetime",
        "relative_continuity": "Relative Continuity",
        "overall_score": "Overall Score"
    })
    df_long = plot_df.melt(var_name="Metric", value_name="Value")
    plt.figure(figsize=(12, 6))
    custom_palette = {
        "Histogram Similarity": "#1f77b4",     # blau
        "Shape Consistency": "#1f77b4",        # blau
        "Area Consistency": "#1f77b4",         # blau
        "Relative ID Lifetime": "#2ca02c",     # grün
        "Relative Continuity": "#2ca02c"       # grün
    }

    # Optional: Filter Overall Score vorher raus
    df_long = df_long[df_long["Metric"] != "Overall Score"]

    sns.boxplot(data=df_long, x="Metric", y="Value", palette=custom_palette)
    plt.title(title)
    plt.ylabel("Metric Value")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, filename))
    plt.close()

Frames processed: 100%|██████████| 10524/10524 [00:45<00:00, 230.13frame/s]
C:\Users\hause\AppData\Local\Temp\ipykernel_27252\3535871327.py:30: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df_long, x="Metric", y="Value", palette=custom_palette)
C:\Users\hause\AppData\Local\Temp\ipykernel_27252\3535871327.py:30: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df_long, x="Metric", y="Value", palette=custom_palette)


# Ball Tracking Evaluation

In [ ]:
import json
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# === Hilfsfunktionen ===
def bbox_area(bbox):
    x1, y1, x2, y2 = bbox
    return max(0, x2 - x1) * max(0, y2 - y1)

def bbox_center(bbox):
    x1, y1, x2, y2 = bbox
    return np.array([(x1 + x2) / 2, (y1 + y2) / 2])

def bbox_shape_ratio(bbox):
    x1, y1, x2, y2 = bbox
    w, h = max(1, x2 - x1), max(1, y2 - y1)
    return w / h

def get_patch(frame, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    h, w = frame.shape[:2]
    x1 = max(0, min(w - 1, x1))
    x2 = max(0, min(w, x2))
    y1 = max(0, min(h - 1, y1))
    y2 = max(0, min(h, y2))
    return frame[y1:y2, x1:x2]

def histogram_similarity(patch1, patch2):
    h1 = cv2.calcHist([patch1], [0], None, [32], [0, 256])
    h2 = cv2.calcHist([patch2], [0], None, [32], [0, 256])
    h1, h2 = cv2.normalize(h1, h1).flatten(), cv2.normalize(h2, h2).flatten()
    return np.sum(np.minimum(h1, h2)) / np.sum(np.maximum(h1, h2))

def compute_fragmentation_metrics(track):
    frame_indices = sorted(t["frame_idx"] for t in track)
    gaps = np.diff(frame_indices)
    fragmentations = np.sum(gaps > 1)
    relative_fragmentation = fragmentations / len(frame_indices) if len(frame_indices) > 0 else 0
    return fragmentations, relative_fragmentation

def batchwise_frame_loader(cap, start_idx, end_idx):
    frames = {}
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_idx)
    for i in range(start_idx, end_idx):
        ret, frame = cap.read()
        if not ret:
            break
        frames[i] = frame
    return frames

def extract_ball_track(tracking_data):
    ball_track = []
    for frame_idx_str, frame_data in tracking_data.items():
        frame_idx = int(frame_idx_str)
        ball_entries = frame_data.get("ball", [])
        if ball_entries:
            bbox = ball_entries[0]["bbox"]
            ball_track.append({
                "frame_idx": frame_idx,
                "bbox": bbox
            })
    return ball_track

# === Hauptfunktion zur Evaluation und Boxplot-Erzeugung ===
def evaluate_ball_metrics_with_all_metrics(tracking_log_path, video_path, output_dir="ball_eval", batch_size=500):
    os.makedirs(output_dir, exist_ok=True)
    excel_path = os.path.join(output_dir, "ball_tracking_metrics.xlsx")
    plot_path = os.path.join(output_dir, "ball_tracking_metrics_boxplot.png")

    with open(tracking_log_path, "r") as f:
        tracking_data = json.load(f)

    total_frames = max(map(int, tracking_data.keys())) + 1
    ball_track = extract_ball_track(tracking_data)
    cap = cv2.VideoCapture(video_path)

    hist_sims, shape_ratios, area_ratios = [], [], []

    frame_progress = tqdm(total=total_frames, desc="Frames processed", unit="frame")
    for batch_start in range(0, total_frames, batch_size):
        batch_end = min(batch_start + batch_size, total_frames)
        frames = batchwise_frame_loader(cap, batch_start, batch_end)
        frame_progress.update(len(frames))

        batch_track = [t for t in ball_track if batch_start <= t["frame_idx"] < batch_end]
        for i in range(1, len(batch_track)):
            f1, f2 = batch_track[i - 1]["frame_idx"], batch_track[i]["frame_idx"]
            b1, b2 = batch_track[i - 1]["bbox"], batch_track[i]["bbox"]
            frame1, frame2 = frames.get(f1), frames.get(f2)
            if frame1 is None or frame2 is None:
                continue

            patch1, patch2 = get_patch(frame1, b1), get_patch(frame2, b2)
            if patch1.size == 0 or patch2.size == 0:
                continue

            try:
                hist_sims.append(histogram_similarity(patch1, patch2))
            except:
                continue

            shape_ratios.append(min(bbox_shape_ratio(b1), bbox_shape_ratio(b2)) / max(bbox_shape_ratio(b1), bbox_shape_ratio(b2)))
            area_ratios.append(min(bbox_area(b1), bbox_area(b2)) / max(bbox_area(b1), bbox_area(b2)))

    cap.release()
    frame_progress.close()

    fragmentations, rel_fragmentation = compute_fragmentation_metrics(ball_track)
    rel_continuity = 1 - rel_fragmentation
    rel_lifetime = len(ball_track) / total_frames if total_frames > 0 else 0

    df = pd.DataFrame({
        "Histogram Similarity": hist_sims,
        "Shape Consistency": shape_ratios,
        "Area Consistency": area_ratios
    })

    # Neue Metriken ergänzen
    df["Relative ID Lifetime"] = rel_lifetime
    df["Relative Continuity"] = rel_continuity

    # Als Excel exportieren
    df.to_excel(excel_path, sheet_name="Ball Metrics", index=False)

    # Für Boxplot: nur ausgewählte Spalten
    df_long = df.melt(var_name="Metric", value_name="Value")
    df_long = df_long[df_long["Metric"].isin([
        "Histogram Similarity", "Shape Consistency", "Area Consistency",
        "Relative ID Lifetime", "Relative Continuity"
    ])]

    custom_palette = {
        "Histogram Similarity": "#1f77b4",
        "Shape Consistency": "#1f77b4",
        "Area Consistency": "#1f77b4",
        "Relative ID Lifetime": "#2ca02c",
        "Relative Continuity": "#2ca02c"
    }

    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df_long, x="Metric", y="Value", palette=custom_palette)
    plt.title("Evaluationsmetriken für den Balltrack")
    plt.ylabel("Metrikwert")
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close()

    return df

# Aufruf der Funktion
evaluate_ball_metrics_with_all_metrics("tracking_log.json", "input.mp4", output_dir="ball_evaluation")


Frames processed: 100%|██████████| 10524/10524 [00:36<00:00, 286.11frame/s]
C:\Users\hause\AppData\Local\Temp\ipykernel_21476\2779455338.py:146: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df_long, x="Metric", y="Value", palette=custom_palette)


,Histogram Similarity,Shape Consistency,Area Consistency,Relative ID Lifetime,Relative Continuity
0,0.252496,0.960424,0.993030,0.822026,0.997573
1,0.365828,0.982033,0.920159,0.822026,0.997573
2,0.179607,0.975889,0.695862,0.822026,0.997573
3,0.614043,0.986618,0.898181,0.822026,0.997573
4,0.525512,0.954291,0.820719,0.822026,0.997573
...,...,...,...,...,...
8625,0.466816,0.960175,0.991631,0.822026,0.997573
8626,0.652047,0.932418,0.968705,0.822026,0.997573
8627,0.653901,0.966497,0.961336,0.822026,0.997573
8628,0.657895,0.941828,0.935344,0.822026,0.997573
